In [1]:
import tensorflow as tf
import far_ho as far

No module named 'experiment_manager'
This is not a required package! However, there might be a couple of functions which could be helpful there!
Might get it at https://github.com/lucfra/ExperimentManager


ordered-set package not found..


In [2]:
def test_problem():
    w = tf.get_variable('w', initializer=[1., 2., -1., -5.])

    e = far.get_hyperparameter('e', initializer=[3., 1.1, 1., 0.1])

    io = e*w**2 + (w - [0., -1., 12., 18.])**2
    oo = (w - [0.1, -1.1, 12.1, 18.1])**2
    return w, e, io, oo

In [3]:
def run_test_reverseHG(inner_optimizer_learning_rate=0.1, iterations=10):
    # initialize everything 
    tf.reset_default_graph()
    try: tf.get_default_session().close()
    except: pass
    ss = tf.InteractiveSession()

    # hypergradient with ReverseHG
    w, e, io, oo = test_problem()

    ho = far.HyperOptimizer()
    ts = ho.minimize(oo, far.GradientDescentOptimizer(0.1), io, 
                     far.GradientDescentOptimizer(inner_optimizer_learning_rate))

    tf.global_variables_initializer().run()
    
    ts(iterations)
    
    return w, e, io, oo, ss

In [4]:
w, e, io, oo, ss = run_test_reverseHG(iterations=100)
ss.run(far.hypergradients())

[array([ 0.       ,  0.2829068, 36.599995 , 51.660515 ], dtype=float32)]

In [5]:
def run_test_implicit_hg(inner_optimizer_learning_rate, iterations=10):
    # initialize everything 
    tf.reset_default_graph()
    try: tf.get_default_session().close()
    except: pass
    ss = tf.InteractiveSession()

    # hypergradient with ReverseHG
    w, e, io, oo = test_problem()

    ho = far.HyperOptimizer(far.ImplicitHG())  # leave all default values!
    ts = ho.minimize(oo, far.GradientDescentOptimizer(0.1), io, 
                     far.GradientDescentOptimizer(inner_optimizer_learning_rate),
                    global_step=tf.Variable(0, trainable=False)) 
    # Note that the global step is necessary since in HOAG you need a decreasing sequence for the tolerance 
    # (even though here we make only one step of hyperparameter optimization....)
    # Note 2: in HOAG paper the optimizer used for soliving the inner problem was BFGS.. which is a bit difficult 
    # to use with tensorflow.. so here I just use plain gradient descent
    
    tf.global_variables_initializer().run()
    # NOTE 2: the condition on the quality of the solution of the inner problem here is not checked! 
    # in HOAG papar that was linked to the tolerance (please see the paper..)
    # Informally speanking, if the inner objective is strongly convex, checking how far the current iterate
    # is from the global minimizer is equivalent to checking that the norm of the gradient is small enough.
    # it would be possible to implement this feature (which is quite important I suppose, especially when you do many 
    # ho steps) by implementing a proper `generator` that checks whether the 
    # condition is verified or not, and if not goes on.. something like this: 
    
#     def check_cond():
#         t = 0
#         if ||_grad|| > ho.hypergradient.tol(global_step):
#             yield t
#             t + =1
    
    # and pass it to `ts` instead of `iterations`.

    ts(iterations)
    return w, e, io, oo, ss


In [6]:
w, e, io, oo, ss  = run_test_implicit_hg(0.1, iterations=10) 

print('inner variable:', w.eval())  # reasonable value
ss.run(far.hypergradients())  # reasonable as well. 
# by increasing the number of iterations from 10 to 100 and 1000 and so on you see that the hypergradient converges

INFO:tensorflow:Optimization terminated with:
  Message: Desired error not necessarily achieved due to precision loss.
  Objective function value: 0.000001
  Number of iterations: 31
  Number of functions evaluations: 156
inner variable: [ 1.0239992e-07 -4.6552294e-01  5.9576735e+00  1.4582811e+01]


[array([5.1199818e-09, 2.8129867e-01, 3.6593979e+01, 9.3255455e+01],
       dtype=float32)]

In [7]:
w, e, io, oo, ss  = run_test_implicit_hg(1, iterations=10) 

print('inner variable:', w.eval())  # not so reasonable value.. inner optimization is not converging...
ss.run(far.hypergradients())  # all 0 
# the problem is this: `Objective function value: 564951424.000000` 

#here the step size of inner optimizer is too high!

INFO:tensorflow:Optimization terminated with:
  Message: Desired error not necessarily achieved due to precision loss.
  Objective function value: 564951424.000000
  Number of iterations: 0
  Number of functions evaluations: 74
inner variable: [ 2.8247526e+08  2.7879388e+05 -4.1333700e+05 -1.1591436e+02]


[array([-0., -0.,  0.,  0.], dtype=float32)]

In [8]:
# as a side-dish.. when increasing the number of iteration the 2 methods (in this case)
# compute (almost) the same hypergrad!
print('IMPLICIT HYPERGRAD')
w, e, io, oo, ss  = run_test_implicit_hg(0.1, iterations=1000) 


print('INNER VAR:', w.eval(), '\n HYPERGRAD', ss.run(far.hypergradients()))

print('\nREVERSE HYPERGRAD')
w, e, io, oo, ss  = run_test_reverseHG(iterations=1000) 


print('INNER VAR:', w.eval(), '\n HYPERGRAD', ss.run(far.hypergradients()))

IMPLICIT HYPERGRAD
INFO:tensorflow:Optimization terminated with:
  Message: Desired error not necessarily achieved due to precision loss.
  Objective function value: 0.000000
  Number of iterations: 32
  Number of functions evaluations: 84
INNER VAR: [ 0.         -0.47619042  5.9999995  16.363632  ] 
 HYPERGRAD [array([ 0.       ,  0.2829068, 36.600002 , 51.660526 ], dtype=float32)]

REVERSE HYPERGRAD
INNER VAR: [ 0.         -0.47619042  5.9999995  16.363632  ] 
 HYPERGRAD [array([ 0.       ,  0.2829068, 36.599995 , 51.660515 ], dtype=float32)]
